# Etherscan - Ethereum Network Metrics History
The purpose of this file is to query the Etherscan API and retrieve Ethereum Netzwork Level metrics 


Outputfile = network_metrics_history.csv

## Table of content

1. Collect the data
  *   1.1 Query API
  *   1.2 Clean the data


In [ ]:
# import Etherscan API Key
f = open("Etherscan_APIKEY.txt", "r")
API_KEY = f.read()

# specify study period
# format 'yyyy-MM-dd'

start = '2016-01-01'
end = '2021-10-31'  

In [ ]:
# function query API endpoints and save results 

def call_api(action, start, end, API_KEY):
    import requests 
    url = "https://api.etherscan.io/api?module=stats&action="+action+"&startdate="+start+"&enddate="+end+"&sort=asc&apikey="+API_KEY # concat API endpoint url

    response = requests.get(url)
    
    # check response status code
    if response.status_code != 200:
        # Raise error code
        raise ApiError('GET /tasks/ {}'.format(resp.status_code))
    
    # store json result in variable
    result = response.json()

    return result

## 1.1 Collect time series data from Etherscan API

* specify list of API actions
* loop through list of API actions and concat a df

In [ ]:
# list of API actions that should be queried

# URLs to API Endpoints actions

# Get Daily Average Block Size
avg_b_size = "dailyavgblocksize"

# Get Daily Uncle Block Count and Rewards
uncle_count_rewards = "dailyuncleblkcount"

# Get Daily Block Count and Rewards
blk_count = "dailyblkcount"

# Get Daily Block Rewards
blk_rewards = "dailyblockrewards"

# Get Daily Average Block Time for A Block to be Included in the Ethereum Blockchain
avg_blk_time = "dailyavgblocktime"

# Get Daily Average Gas Limit
avg_gas_limit = "dailyavggaslimit"

# Get Ethereum Daily Total Gas Used
total_gas_used = "dailygasused"

# Get Daily Average Gas Price
avg_gas_price = "dailyavggasprice"

# Get Daily Network Transaction Fee
txn_fees = "dailytxnfee"

# Get Daily New Address Count
new_addresses = "dailynewaddress"

# Get Daily Network Utilization
network_utilization = "dailynetutilization"

# Get Daily Average Network Hash Rate
avg_hash_rate = "dailyavghashrate"

# Get Daily Transaction Count
txn_count = "dailytx"

# Get Daily Average Network Difficulty
avg_network_difficulty = "dailyavgnetdifficulty"

# Get ETHER Historical Price
eth_price = "ethdailyprice"



Endpoint_action_list = [
    avg_b_size,
    blk_count,
    blk_rewards,
    avg_blk_time,
    avg_gas_limit,
    total_gas_used,
    avg_gas_price,
    txn_fees,
    new_addresses,
    network_utilization,
    avg_hash_rate,
    txn_count,
    avg_network_difficulty,
    eth_price,
    uncle_count_rewards
]

In [ ]:
# get list of responses from API by looping through list of API endpoints

import pandas as pd 
import numpy as np

result_list = []

for action in Endpoint_action_list:
    result_list.append(call_api(action, start, end, API_KEY))

print("API has been called")


In [ ]:
# loop through list of API results and merge dfs based on UTCDate field
i = 0

for result in result_list:

    # create initial df with first query 
    if i == 0:
        # return message from API
        if result["message"] == "OK":
            df_full = pd.DataFrame.from_dict(result["result"])
        else:
            print("Could not import data for: ",Endpoint_action_list[i]+". Message:", result["message"])
            print("Trying with new first df")
            break
    # for following API results: check API message, delete unixTimeStamp, and merge based on based on Timestamp
    if i > 0:
        if result["message"] == "OK":
            df = pd.DataFrame.from_dict(result["result"])
        else:
            print("Could not import data for: ",Endpoint_action_list[i]+". Message:", result["message"])
        df = df.drop("unixTimeStamp", axis= 1)
        df_full = pd.merge(df_full, df, how="outer", on="UTCDate")

    i += 1

print("Data import completed")



## 1.2 Data Cleaning

In [ ]:
# the resulting data frame has some problems
# comma in network difficulty column
# some columns are type object and not numeric 

# Change UTCdate from str to datetime object
df_full['UTCDate'] = pd.to_datetime(df_full['UTCDate'])

# remove comma from networkDifficulty column
df_full["networkDifficulty"] = df_full["networkDifficulty"].str.replace(',', '')


# convert all variables to floats or ints
df_full.iloc[:, 1:] = df_full.iloc[:, 1:].apply(pd.to_numeric)


# add scaled variables
df_full["maxGasPrice_GWei"] = df_full["maxGasPrice_Wei"]*10**-9
df_full["minGasPrice_GWei"] = df_full["minGasPrice_Wei"]*10**-9
df_full["avgGasPrice_GWei"] = df_full["avgGasPrice_Wei"]*10**-9 


# comput avg txn fee
df_full["avgTransactionFee_Eth"] = df_full["transactionFee_Eth"]/df_full["transactionCount"]
df_full["avgTransactionFee_GWei"] = df_full["avgTransactionFee_Eth"]*10**9



df_full.dtypes

In [ ]:
# store the data as csv file
df_full.to_csv("network_metrics_history.csv",index=False, sep=";")

In [2]:
# return min max and avg and media on a given day and compare 
import pandas as pd
test = pd.read_csv(r"path to csv file", sep=",")  
test.head()

,day_,p95_gas_price,p75_gas_price,p50_gas_price,p25_gas_price,p5_gas_price,p95_gas_price_1559,p75_gas_price_1559,p50_gas_price_1559,p25_gas_price_1559,...,p95_total_fee,p75_total_fee,p50_total_fee,p25_total_fee,p5_total_fee,p95_tip,p75_tip,p50_tip,p25_tip,p5_tip
0,2019-05-17,4.800000e+10,2.000000e+10,1.600000e+10,1.000000e+10,3.400000e+09,4.800000e+10,2.000000e+10,1.600000e+10,1.000000e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-07-24,5.400000e+10,2.300000e+10,1.600000e+10,1.265000e+10,1.000000e+09,5.400000e+10,2.300000e+10,1.600000e+10,1.265000e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-06-28,5.550000e+10,3.700000e+10,3.200000e+10,2.700000e+10,2.200000e+10,5.550000e+10,3.700000e+10,3.200000e+10,2.700000e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-12-12,9.600000e+10,4.300100e+10,3.200000e+10,2.420000e+10,1.700000e+10,9.600000e+10,4.300100e+10,3.200000e+10,2.420000e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-11-09,8.433150e+10,5.000000e+10,3.200000e+10,2.500000e+10,1.510000e+10,8.433150e+10,5.000000e+10,3.200000e+10,2.500000e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
